<a href="https://colab.research.google.com/github/kurtsenol/machine-learning/blob/master/TopicModeling/LDA_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import pickle

In [ ]:
df = pd.read_pickle("drive/My Drive/Topic_Modeling/topic_df.pkl")

In [ ]:
df

,Title,Journal,Year,Abstract
0,TWO-DIRECTIONAL CONVERGENCE OF PLATFORM AND PI...,JOURNAL OF SERVICE MANAGEMENT,NaN,PURPOSE THIS ARTICLE EXAMINES THE NEW PHENOMEN...
1,EMPLOYEE WELLNESS ON THE FRONTLINE: AN INTERAC...,JOURNAL OF SERVICE MANAGEMENT,NaN,PURPOSE EMPLOYEE WELLNESS IS VITAL TO CREATING...
2,DEMOGRAPHIC AND ATTITUDINAL ANTECEDENTS OF CON...,JOURNAL OF SERVICE MANAGEMENT,NaN,PURPOSE TO INVESTIGATE IF ONLINE TV CONTENT PL...
3,MORE THAN A FEELING? TOWARD A THEORY OF CUSTOM...,JOURNAL OF SERVICE MANAGEMENT,NaN,PURPOSE RESPONDING TO AN INCREASING CALL FOR A...
4,ACCOUNTABLE ALGORITHMS? THE ETHICAL IMPLICATIO...,JOURNAL OF SERVICE MANAGEMENT,NaN,PURPOSE THE PURPOSE OF THIS STUDY IS TO IDENTI...
...,...,...,...,...
10869,A COMMENTARY ON THE CASE STUDY OF STATE-OWNED ...,CORNELL HOSPITALITY QUARTERLY,2011.0,THE COMPARISON OF STEPHEN RUSHMORE'S SEVEN HOT...
10870,COMMENTARY ON A CASE STUDY OF STATE-OWNED HOTE...,CORNELL HOSPITALITY QUARTERLY,2011.0,THE RECENT LIQUIDITY CRISIS IN THE GLOBAL HOTE...
10871,A MULTIDIMENSIONAL SCALE OF SWITCHING BARRIERS...,CORNELL HOSPITALITY QUARTERLY,2011.0,THIS RESEARCH ATTEMPTED TO DEVELOP A MEASUREME...
10872,A COMPARISON OF COLD-WATER THAWING OPTIONS IN ...,CORNELL HOSPITALITY QUARTERLY,2011.0,CONSIDERING ALL TYPES OF RESTAURANTS IN HONG K...


In [ ]:
papers = list(df.Abstract.values)

In [ ]:
len(papers)

10874

In [ ]:
print(papers[0][:1000])

PURPOSE THIS ARTICLE EXAMINES THE NEW PHENOMENON OF THE CONVERGENCE OF PLATFORM AND PIPELINE BUSINESS MODELS. IT EXAMINES THE POTENTIAL SYNERGIES AND CHALLENGES FOR PLATFORMS TO ADD PIPELINE COMPONENTS AND VICE VERSA FOR PIPELINE BUSINESSES. DESIGN/METHODOLOGY/APPROACH THIS PAPER USES A CONCEPTUAL APPROACH THAT SYNTHESIZES AND INTEGRATES THE LITERATURE FROM SERVICE, HOSPITALITY, AND STRATEGY, AND SUPPLEMENTS THEM WITH TWO ILLUSTRATIVE MINI-CASE STUDIES. FINDINGS WHILE THE EXTANT LITERATURE TYPICALLY FOCUSES ON THE DICHOTOMY BETWEEN INCUMBENT PIPELINE BUSINESSES THAT CREATE VALUE BY CONTROLLING A LINEAR SERIES OF ACTIVITIES AND NETWORK EFFECTS-DRIVEN PLATFORMS, WE DIFFERENTIATE BETWEEN TWO TYPES OF PLATFORM BUSINESS MODELS (I.E. PLATFORMS WITH ASSET CONTROL AND PLATFORMS WITH PEER-PROVIDED ASSETS). FURTHER, WE IDENTIFY THREE COMMON PATHWAYS OF CONVERGENCE; THAT IS, PIPELINES MOVING TOWARDS (1) PLATFORMS WITH ASSET CONTROL AND (2) THOSE WITH PEER-PROVIDED ASSETS, AND (3) PLATFORMS WITH P

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 16.3MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=3b6645dcf09dd5ca40dc40ddc1971f5fd53873e305496f46e24788e27c63b3f1
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=649876a61b751dce4d23675419ebc55c740918a0c7a1ba335daeb09d06633638
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [ ]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=141393a7e7ca6b4549e28953970c796393d782cec9a608730611e6d25c807972
  Stored in directory: /tmp/pip-ephem-wheel-cache-1cj700lm/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
nltk.download('wordnet')
stop_words = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

--2020-06-05 12:22:48--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  17.1MB/s    in 0.9s    

2020-06-05 12:22:49 (17.1 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]



In [ ]:
!unzip mallet-2.0.8.zip

Archive:  mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: mallet-2.0.8/bin/csv2vectors  
  inflating: mallet-2.0.8/bin/mallet  
  inflating: mallet-2.0.8/bin/mallet.bat  
  inflating: mallet-2.0.8/bin/mallethon  
  inflating: mallet-2.0.8/bin/prepend-license.sh  
  inflating: mallet-2.0.8/bin/svmlight2vectors  
  inflating: mallet-2.0.8/bin/text2classify  
  inflating: mallet-2.0.8/bin/text2vectors  
  inflating: mallet-2.0.8/bin/vectors2classify  
  inflating: mallet-2.0.8/bin/vectors2info  
  inflating: mallet-2.0.8/bin/vectors2topics  
  inflating: mallet-2.0.8/bin/vectors2vectors  
  inflating: mallet-2.0.8/build.xml  
   creating: mallet-2.0.8/class/
   creating: mallet-2.0.8/class/cc/
   creating: mallet-2.0.8/class/cc/mallet/
   creating: mallet-2.0.8/class/cc/mallet/classify/
  inflating: mallet-2.0.8/class/cc/mallet/classify/AdaBoost.class  
  in

In [ ]:
mallet_path = '/content/mallet-2.0.8/bin/mallet'

In [ ]:
# future bridges python 2 and 3
from __future__ import print_function

# pandas works with data structures, data manipulation, and analysis specifically for numerical tables, and series like 
# the csv we are using here today
import pandas as pd

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

# Gensim unsupervised topic modeling, natural language processing, statistical machine learning
import gensim
# convert a document to a list of tolkens
from gensim.utils import simple_preprocess
# remove stopwords - words that are not telling: "it" "I" "the" "and" ect.
from gensim.parsing.preprocessing import STOPWORDS
# corpus iterator 
from gensim import corpora, models

# nltk - Natural Language Toolkit
# lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed 
# into present.
# stemmed — words are reduced to their root form.
#import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

# NumPy - multidimensional arrays, matrices, and high-level mathematical formulas
import numpy as np
np.random.seed(2018)

import os
from gensim.models.wrappers import LdaMallet
from pathlib import Path
import codecs
import logging

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

data = df
data_text = data[['Abstract']]
data_text['index'] = data_text.index
documents = data_text

# Create functions to lemmatize stem, and preprocess

# turn beautiful, beautifuly, beautified into stem beauti 
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# parse docs into individual words ignoring words that are less than 3 letters long
# and stopwords: him, her, them, for, there, ect since "their" is not a topic.
# then append the tolkens into a list
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        newStopWords = ['study', 'research', 'paper', 'purpose', 'tourism', 'travel', 'hospitality']
        if token not in gensim.parsing.preprocessing.STOPWORDS and token not in newStopWords and len(token) > 3:
            nltk.bigrams(token)
            result.append(lemmatize_stemming(token))
    return result

# gensim.parsing.preprocessing.STOPWORDS

# look at a random row 4310 and see if things worked out
# note that the document created was already preprocessed

doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

# let’s look at ten rows passed through the lemmatize stemming and preprocess

documents = documents.dropna(subset=['Abstract'])
processed_docs = documents['Abstract'].map(preprocess)
processed_docs[:10]

# we create a dictionary of all the words in the csv by iterating through
# contains the number of times a word appears in the training set.

dictionary_valid = gensim.corpora.Dictionary(processed_docs[20000:])
count = 0
for k, v in dictionary_valid.iteritems():
    print(k, v)
    count += 1
    if count > 30:
        break
        
 # we create a dictionary of all the words in the csv by iterating through
# contains the number of times a word appears in the training set.

dictionary_test = gensim.corpora.Dictionary(processed_docs[:20000])
count = 0
for k, v in dictionary_test.iteritems():
    print(k, v)
    count += 1
    if count > 30:
        break
        
# we want to throw out words that are so frequent that they tell us little about the topic 
# as well as words that are too infrequent >15 rows then keep just 100,000 words

dictionary_valid.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

# we want to throw out words that are so frequent that they tell us little about the topic 
# as well as words that are too infrequent >15 rows then keep just 100,000 words

dictionary_test.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

# the words become numbers and are then counted for frequency
# consider a random row 4310 - it has 8 words word indexed 2 shows up once
# preview the bag of words

bow_corpus_valid = [dictionary_valid.doc2bow(doc) for doc in processed_docs]
bow_corpus_valid[4310]

# the words become numbers and are then counted for frequency
# consider a random row 4310 - it has 8 words word indexed 2 shows up once
# preview the bag of words

bow_corpus_test = [dictionary_test.doc2bow(doc) for doc in processed_docs]
bow_corpus_test[4310]

# same thing in more words

bow_doc_4310 = bow_corpus_test[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary_test[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

#mallet_path = 'C:/mallet/mallet-2.0.8/bin/mallet.bat'


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


original document: 
['THIS', 'PAPER', 'EXPLORES', 'HOW', 'GLOBAL', 'ISSUES', 'SUCH', 'AS', 'CLIMATE', 'CHANGE', 'ARE', 'TAKEN', 'INTO', 'ACCOUNT', 'IN', 'TOURISM', 'STRATEGY', 'TEXTS', 'AND', 'CONTRASTS', 'THESE', 'FINDINGS', 'WITH', 'HOW', 'THE', 'ISSUE', 'IS', 'SEEN', 'AT', 'THE', 'GRASSROOTS', 'LEVEL', 'BY', 'LOCAL', 'BUSINESSES.', 'WE', 'ANALYSE', 'HOW', 'BOTH', 'LEVELS', 'APPROACH', 'ADAPTATION', 'TO', 'CLIMATE', 'CHANGE.', 'USING', 'BOLTANSKI', 'AND', "THEVENOT'S", 'SIX', 'COMMON', '"WORLDS"', 'OF', 'JUSTIFICATION', 'MODEL', 'FOR', 'DEBATES', 'ON', 'PUBLIC', 'ISSUES,', 'WE', 'ANALYSE', 'THE', 'RHETORIC', 'OF', 'NATIONAL,', 'REGIONAL,', 'AND', 'LOCAL', 'TOURISM', 'STRATEGIES', 'IN', 'FINLAND', 'AND', 'THEN', 'EXPLORE', 'HOW', 'THE', 'RHETORIC', 'IS', 'EMPLOYED', 'BY', 'INTERVIEWING', '42', 'LOCAL', 'TOURISM', 'ACTORS.', 'THE', 'STRATEGY', 'ANALYSIS', 'SHOWS', 'THAT', 'STRATEGIC', 'DOCUMENTS', 'DO', 'NOT', 'SIMPLY', 'DESCRIBE', 'SITUATIONS', 'BUT', 'ARE', 'ACTIVE', 'IN', 'CREATING'

TypeError: ignored

In [ ]:

ldamallet_test = gensim.models.wrappers.LdaMallet(mallet_path = '/content/mallet-2.0.8/bin/mallet', corpus=bow_corpus_test, num_topics=10, id2word=dictionary_test)

result = (ldamallet_test.show_topics(num_topics=10, num_words=10,formatted=False))
for each in result:
    print (each)
    
#mallet_path = 'C:/mallet/mallet-2.0.8/bin/mallet.bat'

ldamallet_valid = gensim.models.wrappers.LdaMallet(mallet_path, corpus=bow_corpus_valid, num_topics=20, id2word=dictionary_valid)

result = (ldamallet_valid.show_topics(num_topics=10, num_words=10,formatted=False))
for each in result:
    print (each)
    
# Show Topics
for idx, topic in ldamallet_test.print_topics(-1):
   print('Topic: {} \nWords: {}'.format(idx, topic))
   
# Show Topics
for idx, topic in ldamallet_valid.print_topics(-1):
   print('Topic: {} \nWords: {}'.format(idx, topic))
   
# check out the topics - 30 words - 20 topics

ldamallet_valid.print_topics(idx, 30)

# check out the topics - 30 words - 20 topics

ldamallet_test.print_topics(idx, 30)

# Compute Coherence Score
coherence_model_ldamallet_valid = CoherenceModel(model=ldamallet_valid, texts=processed_docs, dictionary=dictionary_valid, coherence='c_v')
coherence_ldamallet_valid = coherence_model_ldamallet_valid.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet_valid)

# Compute Coherence Score
coherence_model_ldamallet_test = CoherenceModel(model=ldamallet_test, texts=processed_docs, dictionary=dictionary_test, coherence='c_v')
coherence_ldamallet_test = coherence_model_ldamallet_test.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet_test)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


(0, [('develop', 0.03431812068501748), ('sustain', 0.02961086703129203), ('chang', 0.023300905585941), ('industri', 0.01819017304761051), ('polici', 0.016654711736752443), ('sector', 0.014962341970770196), ('stakehold', 0.013303595445171703), ('govern', 0.012698377118264144), ('busi', 0.012003496816999911), ('plan', 0.011364655249708599)])
(1, [('factor', 0.03542773401721927), ('qualiti', 0.028772058374549672), ('restaur', 0.02711119252610368), ('import', 0.0249618367222324), ('result', 0.02316663613604445), ('analysi', 0.020101361665750748), ('food', 0.019930390181351895), ('attribut', 0.0187213775416743), ('measur', 0.017732185381938083), ('evalu', 0.017316968919826585)])
(2, [('social', 0.043018217508917246), ('inform', 0.02950470399540373), ('analysi', 0.02240681780097192), ('onlin', 0.021054269504225218), ('review', 0.019665812845617983), ('gener', 0.01720010533119479), ('relat', 0.015835587580494578), ('method', 0.01572786249491298), ('present', 0.015069542527469898), ('studi', 0

ValueError: ignored

Source: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/